In [1]:
import pandas as pd
import numpy as np
import ordpy
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("cap_52.csv", sep=";")
coluns = ['total_pacotes','total_pacotes_icmp','total_pacotes_udp','total_pacotes_tcp','maior_pacote','menor_pacote','soma_pacotes','total_ips_origem','total_ips_destino','porta_origem_mais_frequente','porta_destino_mais_frequente', 'has_bot']
df

,Unnamed: 0,total_pacotes,total_pacotes_icmp,total_pacotes_udp,total_pacotes_tcp,maior_pacote,menor_pacote,soma_pacotes,total_ips_origem,total_ips_destino,porta_origem_mais_frequente,porta_destino_mais_frequente,has_bot
0,0,3802,6,378,3376,1514,60,2808555,213,226,80.0,1078.0,0
1,1,3259,3,407,2780,1514,60,2097318,203,223,80.0,1078.0,0
2,2,3966,4,327,3587,1514,60,2851044,202,202,80.0,1078.0,0
3,3,3889,9,345,3508,1514,60,2781297,196,208,80.0,1078.0,0
4,4,4077,2,296,3713,1514,60,2923539,184,188,80.0,1078.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,967,3720,2,2962,716,1514,60,3189759,188,192,12649.0,43087.0,0
968,968,3567,5,2860,658,1514,60,2992278,183,183,43087.0,43087.0,0
969,969,3515,6,2418,1009,1514,60,2700171,209,210,12649.0,43087.0,0
970,970,3884,7,2091,1697,1514,60,2892316,211,224,80.0,43087.0,0


In [3]:
matrix = []
label = []
cols = ['total_ips_origem','total_ips_destino', 'total_pacotes', 'total_pacotes_icmp', 'total_pacotes_tcp']

taux = 1
dx = 3

init = 0
end = 48

for i in range (1, end):
    matrix.append([[init, i]])

for i in range(0,925):
    vec = []
    vec.append([init, end])
    
    for col in cols:
        tmp = ordpy.complexity_entropy(df[col][init:end], dx=dx, taux=taux)
        vec.append(tmp[0])
        vec.append(tmp[1])

        tmp = ordpy.fisher_shannon(df[col][init:end], dx=dx, taux=taux)
        vec.append(tmp[1])
    
        edges = ordpy.ordinal_network(df[col][init:end], dx=dx, taux=taux)
    
        tmp = ordpy.complexity_entropy(edges[2], dx=dx, taux=taux)
        vec.append(tmp[0])
        vec.append(tmp[1])
    
        tmp = ordpy.fisher_shannon(edges[2], dx=dx, taux=taux)
        vec.append(tmp[1])
    
        tmp = len(edges[1])
        vec.append(tmp)

        j=0
        indices = []
        for array in edges[1]:
            if(array[0] == array[1]):
                indices.append(j)
            j += 1
    
        prob = 0
        for index in indices:
            tmp = edges[2][index]
            prob += tmp
        vec.append(prob)

    matrix.append(vec)

    bot = 0
    for line in df.iloc[init:end].iterrows():
        if(line[1][4] == 1):
            bot = 1
    label.append(bot)
        
    init = init + 1
    end = end + 1

In [4]:
ft = pd.DataFrame(matrix, columns = ['time',
                                     'IPO_entropy', 'IPO_complexity' , 'IPO_fisher_shanon', 'IPO_edge_entropy', 'IPO_edge_complexity', 'IPO_edge_fisher_shannon', 'IPO_number_nodes', 'IPO_self_transition',
                                     'IPD_entropy', 'IPD_complexity' , 'IPD_fisher_shanon', 'IPD_edge_entropy', 'IPD_edge_complexity', 'IPD_edge_fisher_shannon', 'IPD_number_nodes', 'IPD_self_transition',
                                     'TP_entropy', 'TP_complexity' , 'TP_fisher_shanon', 'TP_edge_entropy', 'TP_edge_complexity', 'TP_edge_fisher_shannon', 'TP_number_nodes', 'TP_self_transition',
                                     'TP_ICMP_entropy', 'TP_ICMP_complexity' , 'TP_ICMP_fisher_shanon', 'TP_ICMP_edge_entropy', 'TP_ICMP_edge_complexity', 'TP_ICMP_edge_fisher_shannon', 'TP_ICMP_number_nodes', 'TP_ICMP_self_transition',
                                     'TP_TCP_entropy', 'TP_TCP_complexity' , 'TP_TCP_fisher_shanon', 'TP_TCP_edge_entropy', 'TP_TCP_edge_complexity', 'TP_TCP_edge_fisher_shannon', 'TP_TCP_number_nodes', 'TP_TCP_self_transition'])
ft
ft.to_csv('po_52.csv',sep=';')
ft[0:50]

,time,IPO_entropy,IPO_complexity,IPO_fisher_shanon,IPO_edge_entropy,IPO_edge_complexity,IPO_edge_fisher_shannon,IPO_number_nodes,IPO_self_transition,IPD_entropy,...,TP_ICMP_number_nodes,TP_ICMP_self_transition,TP_TCP_entropy,TP_TCP_complexity,TP_TCP_fisher_shanon,TP_TCP_edge_entropy,TP_TCP_edge_complexity,TP_TCP_edge_fisher_shannon,TP_TCP_number_nodes,TP_TCP_self_transition
0,"[0, 1]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[0, 2]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[0, 3]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[0, 4]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[0, 5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"[0, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"[0, 7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"[0, 8]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"[0, 9]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"[0, 10]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
import numpy as np

po = pd.read_csv("po_52.csv", sep=";")
early_warning = pd.read_csv("cap_52.csv", sep=";")


df = po[['IPO_entropy', 'IPO_complexity',
       'IPO_fisher_shanon', 'IPO_edge_entropy', 'IPO_edge_complexity',
       'IPO_edge_fisher_shannon', 'IPO_number_nodes', 'IPO_self_transition',
       'IPD_entropy', 'IPD_complexity', 'IPD_fisher_shanon',
       'IPD_edge_entropy', 'IPD_edge_complexity', 'IPD_edge_fisher_shannon',
       'IPD_number_nodes', 'IPD_self_transition', 'TP_entropy',
       'TP_complexity', 'TP_fisher_shanon', 'TP_edge_entropy',
       'TP_edge_complexity', 'TP_edge_fisher_shannon', 'TP_number_nodes',
       'TP_self_transition',
        'TP_ICMP_entropy', 'TP_ICMP_complexity' , 'TP_ICMP_fisher_shanon', 'TP_ICMP_edge_entropy', 'TP_ICMP_edge_complexity', 'TP_ICMP_edge_fisher_shannon', 'TP_ICMP_number_nodes', 'TP_ICMP_self_transition',
        'TP_TCP_entropy', 'TP_TCP_complexity' , 'TP_TCP_fisher_shanon', 'TP_TCP_edge_entropy', 'TP_TCP_edge_complexity', 'TP_TCP_edge_fisher_shannon', 'TP_TCP_number_nodes', 'TP_TCP_self_transition']]
label = early_warning['has_bot']

init = 48
train = 324
end = 778

x_train = df[init:train]
x_test = df[train:end]
y_real = label[train:end]

clf = OneClassSVM(gamma='auto', kernel='poly', nu=0.1).fit(x_train)
y_test = clf.predict(x_test)
y_test_final = [0 if i == 1 else 1 for i in y_test]
acc = accuracy_score(y_real, y_test_final)
print(acc)
print(confusion_matrix(y_real, y_test_final))
print(classification_report(y_real, y_test_final, digits=4))

0.8964757709251101
[[404  26]
 [ 21   3]]
              precision    recall  f1-score   support

           0     0.9506    0.9395    0.9450       430
           1     0.1034    0.1250    0.1132        24

    accuracy                         0.8965       454
   macro avg     0.5270    0.5323    0.5291       454
weighted avg     0.9058    0.8965    0.9011       454

